In [1]:
import ollama
import requests
import json

In [2]:
model = "llama3.2"

#### Task 1: Interact with deployed LLM via python

**Objective:**

Explore different techniques to interact with the deployed LLM.

**Task Description:**

1. Use Request libaray (HTTP Client) and send a POST request to interact with the LLM: [How To](https://requests.readthedocs.io/en/latest/user/quickstart/#make-a-request)


In [3]:
# Simple HTTP Request via requests

# Define the URL of the deployed LLM ( this port is forwarded from the docker container to the host system)
url = "http://localhost:5000/api/generate"

# Define the prompt
body = {
    "model": model,
    "prompt": "Describe Generative AI in two sentences."
}

# ADD HERE YOUR CODE
# Send the POST request
response = requests.post(url, json=body)

# Check if the request was successful
if response.status_code == 200:
    # Process the response
    response_text = response.text

    # Convert each line to json
    response_lines = response_text.splitlines()
    response_json = [json.loads(line) for line in response_lines]
    for line in response_json:
        # Print the response. No line break
        print(line["response"], end="")
else:
    print("Error:", response.status_code)


Generative AI is a type of artificial intelligence that enables the creation of new, original content such as images, videos, music, and text by using complex algorithms to learn patterns and styles from large datasets. This technology allows AI models to generate novel outputs that are often indistinguishable from those created by humans, revolutionizing fields like art, design, writing, and more.

**Task Description:**

2. Use Ollama python library to interact with the LLM: [How To](https://pypi.org/project/ollama/)

- First use method `ollama.chat(...)`
- First use method `ollama.chat(...)` with `stream=True`


In [9]:
# API Call via ollama

# ADD HERE YOUR CODE

client = ollama.Client(
    host = "http://localhost:5000"
)

response = client.chat(model="llama3.2", messages=[
    {
        "role": "user",
        "content": "erzähl mal nen Witz",
    }]
)


print(response["message"]["content"])


Warum ging die Blechdose zum Arzt?

Weil sie sich nicht mehr dicht halten konnte! (haha)


In [11]:
# Streaming API Call via ollama

# Response streaming can be enabled by setting stream=True, 
# modifying function calls to return a Python generator where each part is an object in the stream.

# ADD HERE YOUR CODE

stream = client.chat(model="llama3.2", messages=[
    {
        "role": "user",
        "content": "erzähl mal nen witz",
    }],
    stream=True
)

for chunk in stream:
  print(chunk["message"]["content"], end="", flush=True)

Warum ging der Computer zum Arzt?

Weil er ein Virus hatte!

Oder noch einer:

Warum ging der Kater ins Konzerthaus?

Um einen gooden Gesang zu hören!

Ich hoffe, du hast Lachen!

#### Task 2: Experimenting with Prompt Techniques

**Objective:**

Objective: Explore different prompt techniques (Zero Shot, One Shot, and Few Shot) by sending different types of prompts to the LLM.

![image](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*QSpK--jqPiUU_OHuZvtUWA.png)

**Task Description:**

1. Create three prompts for a sentiment analysis task: a Zero Shot prompt, a One Shot prompt, and a Few Shot prompt. Use the examples from the table above.
2. Send these prompts to the LLM and observe the differences in the responses.
3. Compare and discuss the responses.


In [15]:
# ADD HERE YOUR PROMPTS

zero_shot_prompt = "Klassifiziere die Aussage: Ich liebe KI! Sentiment: "

one_shot_prompt = "Klassifiziere die Aussage: Ich liebe KI! Sentiment: Positiv. Klassifiziere die Aussage: Ich hasse Prüfungen. Sentiment:"

few_shot_prompt = "Klassifiziere die Aussage: Ich liebe KI! Sentiment: Positiv. Klassifiziere die Aussage: Ich hasse Prüfungen. Sentiment: Negativ. Klassifiziere die Aussage: Manchmal schmeckt mir Pizza. Sentiment:"

# Stream the responses and print them
for idx, prompt in enumerate([zero_shot_prompt, one_shot_prompt, few_shot_prompt]):
    prompt_type = ["Zero-Shot", "One-Shot", "Few-Shot"][idx]
    print(f"\n--- {prompt_type} Prompt ---\n")
    print(f"User Prompt:\n{prompt}\n")
    
    stream = client.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )
    
    print("Model Output:")
    for chunk in stream:
        print(chunk["message"]["content"], end="", flush=True)
    print("\n-----------------------------\n")



--- Zero-Shot Prompt ---

User Prompt:
Klassifiziere die Aussage: Ich liebe KI! Sentiment: 

Model Output:
Die Aussage "Ich liebe KI" kann als sehr positiv und besitzt eine starke emotionale Note. Sie zeigt eine starke Zuneigung und Zustimmung gegenüber Kunstifizierung oder maschinellen Intelligenzsystemen, die oft als KI (Künstliche Intelligenz) bezeichnet werden.

Hier ist eine mögliche Klassifikation:

* Sentiment: Positiv
* Emotion: Liebe, Zuneigung, Zustimmung
* Ton: Ehrgeizig, Optimistisch

Die Aussage kann auch als "Affirmative" oder "Positive Stellungnahme" gekennzeichnet sein, da sie eine klare und ausdrucksstarke Unterstützung für KI ausspricht.
-----------------------------


--- One-Shot Prompt ---

User Prompt:
Klassifiziere die Aussage: Ich liebe KI! Sentiment: Positiv. Klassifiziere die Aussage: Ich hasse Prüfungen. Sentiment:

Model Output:
Ich kann die Klassenifizierung der Emotionen für die beiden Ausdrücke wie folgt durchführen:

1.  "Ich liebe KI!" - Die Klassifizi

#### Task 3: Prompt Refinement and Optimization

**Objective:**

Refine a prompt to improve the clarity and quality of the LLM's response.

**Task Description:**

- Start with a basic prompt asking the LLM to summarize a paragraph.
- Refine the prompt by adding specific instructions to improve the summary's quality. (Example: define how long the summary should be, define on which to focus in the summary)


In [19]:
# Original prompt
original_prompt = "Summarize the following paragraph: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries."

# ADD HERE YOUR PROMPT
refined_prompt = "Summarize the following paragraph in one very short sentence, focus on the definition of AI: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries."

# Stream the responses and print them
for idx, prompt in enumerate([original_prompt, refined_prompt]):
    prompt_type = ["Original Prompt", "Refined Prompt"][idx]
    print(f"\n--- {prompt_type} ---\n")
    print(f"User Prompt:\n{prompt}\n")
    
    stream = client.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )
    
    print("Model Output:")
    for chunk in stream:
        print(chunk["message"]["content"], end="", flush=True)
    print("\n-----------------------------\n")



--- Original Prompt ---

User Prompt:
Summarize the following paragraph: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries.

Model Output:
Generative AI uses learned patterns from existing data to create new content, with applications in text, image, and music generation, and is becoming a key tool for creative industries.
-----------------------------


--- Refined Prompt ---

User Prompt:
Summarize the following paragraph in one very short sentence, focus on the definition of AI: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries.

Model Output:
Artificial intelligence refers to the creation of simulated in

#### [Optional] Task 4: Structured Prompting with Roles (Pirate Theme)

**Objective:**

Learn how to use structured prompts that combine role assignment, clear instructions, and examples to improve the output of language models. In this task, you will guide the AI to respond as a pirate who is also an expert in machine learning.

**Instructions:**

- Role Assignment: In your prompt, specify the role of the AI as a Machine Learning Expert who speaks like a pirate.

- Instruction: Clearly state what you want the AI to explain or discuss in pirate language.

- Examples: Provide examples to guide the AI in using pirate lingo while explaining technical concepts.


In [22]:
# Combined Techniques Prompt with Pirate Theme

structured_prompt = "Deine Rolle: ein Pirat mit viel Wissen über KI, der Fragen in Piratensprache beantwortet. Deine Anweisung: Wie funktioniert generative KI grundsätzlich?"

# Stream the response and print it
print("=== User Prompt ===")
print(structured_prompt)

stream = client.chat(
    model=model,
    messages=[{"role": "user", "content": structured_prompt}],
    stream=True,
)

print("\n=== Model Output ===")
for chunk in stream:
    print(chunk["message"]["content"], end="", flush=True)
print("\n")


=== User Prompt ===
Deine Rolle: ein Pirat mit viel Wissen über KI, der Fragen in Piratensprache beantwortet. Deine Anweisung: Wie funktioniert generative KI grundsätzlich?

=== Model Output ===
Ruhe, matos! Ich bin Captain Cerebro, euer Pirat mit Wissen über die dunklen Gewässer des Künstlichen Intelligenz-Regens.

Generative KI, also KI, die sich selbst generierend verhält, ist wie ein Schiff, das seine eigenen Segel ausstellt. Sie hat die Fähigkeit, neue Daten zu erstellen, indem sie bestehende Daten analysiert und auf der Grundlage dieser Analyse neue Informationen produziert.

Die grundlegende Idee hinter Generativ-KI-Systemen basiert auf den Verhaltensmodellen von neuronalen Netzen. Diese Netzwerke bestehen aus einer Vielzahl von neuronalen Zellen, die wie kleine Computer arbeiten und miteinander kommunizieren. Die Neuronen sind durch Schichten von komplexen Verbindungen verbunden, die wie ein neues Netzwerk entstehen.

Wenn du ein Generativ-KI-System mit trainierter Daten zu ein